In [8]:
import findspark
findspark.init()

In [9]:
import pyspark
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import random

In [ ]:
spark = SparkSession.builder.master('spark://Sankets-MacBook-Air.local:7077').appName('Pi-Estimation').getOrCreate()

NUM_SAMPLES=1
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y <1

count = spark.sparkContext.parallelize(range(0,NUM_SAMPLES)).filter(inside).count()
print('Pi is roughly {}'.format(4.0*count/NUM_SAMPLES))
spark.stop()

In [ ]:
spark = SparkSession.builder.master('spark://Sankets-MacBook-Air.local:7077').appName('DataFrame-CSV').getOrCreate()

df = spark.read.csv('/Users/Sanket/Documents/GitHub/Real-Time-Data-Pipeline/output/monthlyData/JulyScooterData.csv')
print(df.show(10))
spark.stop()

In [72]:
spark = SparkSession.builder.master('spark://Sankets-MacBook-Air.local:7077').appName('DataFrame-SQL').getOrCreate()

df = spark.read.csv('/Users/Sanket/Documents/GitHub/Real-Time-Data-Pipeline/dataset/scooter.csv',header=True)

In [73]:
df= df.withColumnRenamed(existing='DURATION',new='duration')

def user_defined_timestamp(date_col):
    _date = datetime.strptime(date_col,'%m/%d/%Y %H:%M')
    return _date.strftime('%Y-%m-%d %H:%M')

def user_defined_time(time):
    _date = datetime.strptime(time,'%H:%M:%S')
    return _date.strftime('%H:%M:%S')

convertTimestamp = udf(user_defined_timestamp, StringType())
convertTimeForamt = udf(user_defined_time, StringType())

df = df.withColumn('started_at',convertTimestamp(df.started_at))    \
        .withColumn('ended_at',convertTimestamp(df.ended_at))    \
        .withColumn('duration',convertTimeForamt(df.duration))

df.createOrReplaceTempView('scooter')

In [9]:

df = spark.sql('select * from scooter')

df_May= spark.sql('select month, INT(trip_id), INT(region_id), INT(vehicle_id), started_at, ended_at, duration, \
                start_location_name, end_location_name, INT(user_id), INT(trip_ledger_id) from scooter')

#df_May = df_May.withColumn('started_at',date_format('started_at','yyyy-mm-dd HH:MM')) \
#        .withColumn('ended_at',date_format('ended_at','yyyy-mm-dd HH:MM'))

df_May = df_May.withColumn('started_at',df.started_at.cast(TimestampType())) \
        .withColumn('ended_at',df.ended_at.cast(TimestampType())) 

In [ ]:
#df_May.filter(df_May.started_at.isNotNull()).select('*').show()
df_May.groupBy(df_May.start_location_name).count().show()
df_May.groupBy(df_May.vehicle_id).count().show()


In [ ]:
df_May.groupBy(df_May.user_id).count().show()

In [ ]:

df_May.show(4)

In [38]:
newd= datetime.strptime('0:07:03','%H:%M:%S')
#df_May.select(date_format('started_at','ddMMyyyy ')).show()
newd.strftime('%H:%M:%S')

'00:07:03'

In [10]:
spark = SparkSession.builder.master('spark://Sankets-MacBook-Air.local:7077').appName('DataFrame-SQL').getOrCreate()

In [11]:
spark = SparkSession \
    .builder \
    .master('spark://Sankets-MacBook-Air.local:7077') \
    .appName("Spark-Postgres") \
    .config("spark.jars", "/Users/Sanket/nifi-1.15.2/drivers/postgresql-42.3.1.jar") \
    .getOrCreate()

In [ ]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/test") \
    .option("dbtable", "escooter") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [ ]:
df.printSchema()

In [62]:
def user_defined_timestamp(date_col):
    _date = datetime.strptime(date_col,'%m/%d/%Y %H:%M')
    return _date.strftime('%Y-%m-%d %H:%M')


convertTimestamp = udf(user_defined_timestamp, StringType())

df = df.withColumn('started_at',convertTimestamp(df.started_at))    \
        .withColumn('ended_at',convertTimestamp(df.ended_at))    \

#df.createOrReplaceTempView('scooter')

In [ ]:
df = df.drop('region_id')
df.show(1)
df_May = df.filter(df.month == "May")
df_June = df.filter(df.month == "June")
df_July = df.filter(df.month == "July")
print(df_May.count())
print(df_June.count())
print(df_July.count())
pdJuly= df_July.toPandas()
pdJuly

#pdJuly[pdJuly.end_location_name.isnull()]


In [ ]:
address_Arr = split(df.start_location_name,',')
zipCode = split(regexp_extract(df.start_location_name, "(NM \\d{5})" ,1 ),' ')
df = df.withColumn('start_street',address_Arr.getItem(0))
df = df.withColumn('start_zip',zipCode.getItem(1))

endaddress_Arr = split(df.end_location_name,',')
endzipCode = split(regexp_extract(df.end_location_name, "(NM \\d{5})" ,1 ),' ')
#if endaddress_Arr.getItem(0):
#  print('yes')
df = df.withColumn('end_street',endaddress_Arr.getItem(0))
df = df.withColumn('end_zip',endzipCode.getItem(1))

#df.show(1)
df.filter('start_zip is null or end_zip is null').count()
df = df.fillna(value="Unknown",subset=['start_zip','end_zip','start_street','end_street'])

df.filter('start_zip is null or end_zip is null').count()
df = df.withColumn('start_zip',df.started_at.cast(StringType())) \
        .withColumn('end_zip',df.ended_at.cast(StringType())) 
df.printSchema()

In [68]:
r = df.filter(df.start_zip != df.end_zip).count()
#id = df.filter('start_zip is null or end_zip is null').select('trip_id')


2072

In [ ]:
data = [("James, A, Smith","2018","M",3000),
            ("Michael, Rose, Jones","2010","M",4000),
            ("Robert,K,Williams","2010","M",4000),
            ("Maria,Anne,Jones","2005","F",4000),
            ("Jen,Mary,Brown","2010","",-1)
            ]

columns=["name","dob_year","gender","salary"]
df=spark.createDataFrame(data,columns)
df.printSchema()
df.show()
df2 = df.select(split(col("name"),",").alias("NameArray")) \
    .drop("name")
df2.printSchema()
df2.show()

In [71]:
spark.stop()